In [ ]:
%%capture
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip

In [ ]:
!unzip '/content/sentiment labelled sentences.zip'

Archive:  /content/sentiment labelled sentences.zip
   creating: sentiment labelled sentences/
  inflating: sentiment labelled sentences/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/sentiment labelled sentences/
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('/content/sentiment labelled sentences/amazon_cells_labelled.txt',delimiter='\t',names=['review','labelled_sentiment'])
df2 = pd.read_csv('/content/sentiment labelled sentences/imdb_labelled.txt',delimiter='\t',names=['review','labelled_sentiment'])
df3 = pd.read_csv('/content/sentiment labelled sentences/yelp_labelled.txt',delimiter='\t',names=['review','labelled_sentiment'])

In [ ]:
df = pd.concat([df1,df2,df3],axis=0,ignore_index=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review              2748 non-null   object
 1   labelled_sentiment  2748 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


In [ ]:
df

,review,labelled_sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2743,I think food should have flavor and texture an...,0
2744,Appetite instantly gone.,0
2745,Overall I was not impressed and would not go b...,0
2746,"The whole experience was underwhelming, and I ...",0


# Sentiment Score calculation and labelling with transformers

In [ ]:
from transformers import pipeline

In [ ]:
distilbert_classifier = pipeline('sentiment-analysis',truncation=True)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
print(df['review'][1])
test =distilbert_classifier(df['review'][1])
print(test)

Good case, Excellent value.
[{'label': 'POSITIVE', 'score': 0.9998685121536255}]


In [ ]:
test[0]['label']

'POSITIVE'

In [ ]:
test[0]['score']

0.9998685121536255

In [ ]:
df.head()

,review,labelled_sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
df['distilbert_sentiment'] = df['review'].apply(lambda x: distilbert_classifier(x))

In [23]:
df.head()

,review,labelled_sentiment,distilbert_sentiment
0,So there is no way for me to plug it in here i...,0,"[{'label': 'NEGATIVE', 'score': 0.999408602714..."
1,"Good case, Excellent value.",1,"[{'label': 'POSITIVE', 'score': 0.999868512153..."
2,Great for the jawbone.,1,"[{'label': 'POSITIVE', 'score': 0.999779641628..."
3,Tied to charger for conversations lasting more...,0,"[{'label': 'NEGATIVE', 'score': 0.999404191970..."
4,The mic is great.,1,"[{'label': 'POSITIVE', 'score': 0.999868988990..."


In [24]:
roberta_classifier = pipeline('sentiment-analysis',model="siebert/sentiment-roberta-large-english",truncation = True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
df['roberta_sentiment'] = df['review'].apply(lambda x:roberta_classifier(x))

In [26]:
bert_classifier = pipeline('sentiment-analysis',model="barissayil/bert-sentiment-analysis-sst",truncation = True)

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at barissayil/bert-sentiment-analysis-sst were not used when initializing BertForSequenceClassification: ['cls_layer.bias', 'cls_layer.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at barissayil/bert-sentiment-analysis-sst and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [27]:
df['bert_sentiment'] = df['review'].apply(lambda x: bert_classifier(x))

In [28]:
df.head()

,review,labelled_sentiment,distilbert_sentiment,roberta_sentiment,bert_sentiment
0,So there is no way for me to plug it in here i...,0,"[{'label': 'NEGATIVE', 'score': 0.999408602714...","[{'label': 'NEGATIVE', 'score': 0.999331653118...","[{'label': 'LABEL_0', 'score': 0.5284364819526..."
1,"Good case, Excellent value.",1,"[{'label': 'POSITIVE', 'score': 0.999868512153...","[{'label': 'POSITIVE', 'score': 0.998826086521...","[{'label': 'LABEL_1', 'score': 0.5199737548828..."
2,Great for the jawbone.,1,"[{'label': 'POSITIVE', 'score': 0.999779641628...","[{'label': 'POSITIVE', 'score': 0.998665452003...","[{'label': 'LABEL_1', 'score': 0.5418449640274..."
3,Tied to charger for conversations lasting more...,0,"[{'label': 'NEGATIVE', 'score': 0.999404191970...","[{'label': 'NEGATIVE', 'score': 0.999490618705...","[{'label': 'LABEL_0', 'score': 0.5166104435920..."
4,The mic is great.,1,"[{'label': 'POSITIVE', 'score': 0.999868988990...","[{'label': 'POSITIVE', 'score': 0.998643219470...","[{'label': 'LABEL_1', 'score': 0.5366623997688..."
